In [1]:
import numpy as np
from pathlib import Path
import pymc as pm
import arviz as az
import multiprocessing as mp
from estival.wrappers import pymc as epm
from estival.sampling import tools as esamp
from estival.wrappers import nevergrad as eng
from estival.utils.parallel import map_parallel
import nevergrad as ng

from tbdynamics.constants import BURN_IN, OPTI_DRAWS
from tbdynamics.inputs import load_targets
from tbdynamics.utils import get_target_from_name
from autumn.infrastructure.remote import springboard
from tbdynamics.calib_utils import get_bcm
import pandas as pd
from tbdynamics.calib_utils import get_targets
from tbdynamics.calib_utils import plot_spaghetti

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
params = {
    "start_population_size": 2300000.0,
    "seed_time": 1810.0,
    "seed_num": 10.0,
    "seed_duration": 3.0,
}

bcm = get_bcm(params)

In [3]:
def calibrate(out_path, bcm, draws, tune):
    lhs_samples = bcm.sample.lhs(8, ci=0.67)
    lhs_lle = esamp.likelihood_extras_for_samples(lhs_samples, bcm)
    lhs_sorted = lhs_lle.sort_values("loglikelihood", ascending=False)
    start_lhs = lhs_samples[lhs_sorted.index].iloc[0:8]

    def optimize_ng(idx_sample):
        idx, sample = idx_sample
        opt = eng.optimize_model(
            bcm,
            budget=100,
            opt_class=ng.optimizers.TwoPointsDE,
            obj_function=bcm.logposterior,
            suggested=sample,
            num_workers=4,
            ci=0.67
        )
        rec = opt.minimize(OPTI_DRAWS)
        return idx, rec.value[1]

    opt_samples = map_parallel(
        optimize_ng, start_lhs.iterrows(), n_workers=2, mode="process"
    )
    opt_samples = bcm.sample.convert(opt_samples)
    init_samps = opt_samples.iloc[0:8].convert("list_of_dicts")
    n_chains = 8
    n_samples = 200
    with pm.Model() as pm_model:
        variables = epm.use_model(bcm)
        idata_raw = pm.sample(
            step=[pm.DEMetropolisZ(variables)],
            draws=draws,
            tune=tune,
            cores=8,
            discard_tuned_samples=False,
            chains=n_chains,
            progressbar=True,
            initvals=init_samps,
        )
    idata_raw.to_netcdf(str(out_path / "calib_full_out.nc"))

    burnt_idata = idata_raw.sel(draw=np.s_[BURN_IN:])
    idata_extract = az.extract(burnt_idata, num_samples=n_samples)
    bcm.sample.convert(idata_extract).to_hdf5(out_path / "calib_extract_out.h5")

    spaghetti_res = esamp.model_results_for_samples(idata_extract, bcm)
    spaghetti_res.results.to_hdf(str(out_path / "results.hdf"), "spaghetti")

    like_df = esamp.likelihood_extras_for_idata(idata_raw, bcm)
    like_df.to_hdf(str(out_path / "results.hdf"), "likelihood")


def run_calibration(bridge: springboard.task.TaskBridge, bcm, draws, tune):
    import multiprocessing as mp
    mp.set_start_method("forkserver")
    idata_raw = calibrate(bridge.out_path, bcm, draws, tune)
    bridge.logger.info("Calibration complete")

In [4]:

OUT_PATH = Path.cwd() / 'runs/r0504'
draws= 100
tune = 10
calibrate(OUT_PATH,bcm, draws, tune)


Only 100 samples in chain.
Multiprocess sampling (8 chains in 8 jobs)
DEMetropolisZ: [contact_rate, rr_infection_latent, rr_infection_recovered, progression_multiplier, smear_positive_death_rate, smear_negative_death_rate, smear_positive_self_recovery, smear_negative_self_recovery, screening_scaleup_shape, screening_inflection_time, screening_end_asymp, detection_reduction]


Sampling 8 chains for 10 tune and 100 draw iterations (80 + 800 draws total) took 84 seconds.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


In [5]:
draws = 50000
tune = 100000

commands = [
    'git clone --branch tb-covid https://github.com/longbui/tbdynamics.git',
    'pip install -e ./tbdynamics',
]

mspec = springboard.EC2MachineSpec(8, 2, 'compute')
run_str = f'd{int(draws / 1000)}k-t{int(tune / 1000)}k-b{int(BURN_IN / 1000)}k'
tspec_args = {'bcm': bcm,'draws': draws, 'tune': tune}
tspec = springboard.TaskSpec(run_calibration, tspec_args)
run_path =  springboard.launch.get_autumn_project_run_path('tbdynamics', 's1004_calibration', run_str)
runner = springboard.launch.launch_synced_autumn_task(tspec, mspec, run_path, branch=None, extra_commands=commands)


SSH connection still waiting, retrying 1/5
SSH connection still waiting, retrying 2/5


In [ ]:
# # from autumn.infrastructure.remote import springboard
# rts = springboard.task.RemoteTaskStore()
# rts.cd('projects/tbdynamics/s06043_calibration')
# rts.ls()
# mt = rts.get_managed_task('2024-04-05T1445-d500k-t100k-b25k')
mt.get_log("crash")

In [7]:
print(runner.tail())

  "accept": np.exp(accept)
/home/ubuntu/miniconda/envs/autumn310/lib/python3.10/site-packages/pymc/step_methods/metropolis.py:1020: RuntimeWarning: overflow encountered in ex
  "accept": np.exp(accept)
/home/ubuntu/miniconda/envs/autumn310/lib/python3.10/site-packages/pymc/step_methods/metropolis.py:1020: RuntimeWarning: overflow encountered in ex
  "accept": np.exp(accept)
/home/ubuntu/miniconda/envs/autumn310/lib/python3.10/site-packages/pymc/step_methods/metropolis.py:1020: RuntimeWarning: overflow encountered in ex
  "accept": np.exp(accept)
/home/ubuntu/miniconda/envs/autumn310/lib/python3.10/site-packages/pymc/step_methods/metropolis.py:1020: RuntimeWarning: overflow encountered in ex
  "accept": np.exp(accept)
 |█--------------------------| 5.36% [257463/4800000 17:42<5:12:27 Sampling 8 chains, 0 divergences]



In [8]:
runner.terminate()

CommandResult(complete=True, exit_status=None, stdout='', stderr='', host_output=	host=13.239.38.45
	alias=None
	exit_code=None
	channel=<ssh2.channel.Channel object at 0x0000020B45A9C8E0>
	exception=None
	encoding=utf-8
	read_timeout=1.0)

In [ ]:
files =runner.get_managed_task()

In [ ]:
files.download_all()